In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split # to split the dataset for training and testing
import random
from sklearn.metrics import precision_recall_fscore_support as score
import math

Using TensorFlow backend.


In [2]:
df = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/ratings.csv')
movies = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/movies.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
new_df = pd.concat([movies.drop('genres', 1), movies['genres'].str.get_dummies(sep="|")], 1)

In [6]:
df = pd.merge(df, new_df, on='movieId')

In [7]:
def f(user):
    profile_1 = df[df['userId']==user]
    profile_1.drop(['movieId','title','timestamp','userId'], axis=1, inplace=True)
    profile_1["sum"] = profile_1.sum(axis=1)
    profile1_new = profile_1.loc[:,"rating":"Western"].div(profile_1["sum"], axis=0)
    return profile1_new

In [8]:
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]

    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.rating + w2*row.Action + w3*row.Adventure + w4*row.Animation + w5*row.Children + w6*row.Comedy + w7*row.Crime + w8*row.Documentary + w9*row.Drama + w10*row.Fantasy + w12*row.Horror + w13*row.IMAX + w14*row.Musical + w15*row.Mystery + w16*row.Romance + w18*row.Thriller + w19*row.War + w20*row.Western)

    return ans

In [9]:
lb = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
ub = [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255]

In [10]:
# implementation of PSO
# checking and defining weight inside the function; a random number between 0.5 and 1
def pso(fitness_func, lbound, ubound, swarm_size=40, max_iter=10, dimensions=20, phip=1.494, phig=1.494):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]

    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/100)
        v_max.append((ubound[i]-lbound[i])/100)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                r1=random.uniform(0,1)
                r2=random.uniform(0,1)
                weight = random.uniform(0.5, 1)
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
                if v[i][j] > v_max[j]:
                    v[i][j] = v_max[j]
                
                if v[i][j] < v_min[j]:
                    v[i][j] = v_min[j]
                                 
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [11]:
# function to predict weights of a particular user using Particle Swarm Optimization
def predict_weights_PSO(df, user):
    user_df = df[df['userId']==user]
    # estimating weights using pso
    xopt, fopt = pso(fitness_func, lb, ub, swarm_size=40,max_iter=40)
    return xopt

In [12]:
def predict_weights_all_pso(df):
    user_weight_dict = {}
    i=0
    for user in df.userId.unique():
        i+=1
        xopt = predict_weights_PSO(df, user)
        user_weight_dict[user] = xopt
        print(i)
    return user_weight_dict

user_weight_dict = predict_weights_all_pso(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
# functions for finding similarity between two users based on their preferences to individual criteria
# based on euclidean distance
# where weights is a list of individual weights of user1
def euclidean_similarity_weighted(df, user1, user2, weights):
    s = pd.merge(df[df['userId']==user1], df[df['userId']==user2], how="inner", on="movieId")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    for index, row in s.iterrows():
           
        tempvar = (w1*((row.rating_x - row.rating_y)**2) + \
                w2*((row.Action_x - row.Action_y)**2) + \
                w3*((row.Adventure_x - row.Adventure_y)**2) + \
                w4*((row.Animation_x - row.Animation_y)**2) + \
                w5*((row.Children_x - row.Children_y)**2) + \
                w6*((row.Comedy_x - row.Comedy_y)**2) + \
                w7*((row.Crime_x - row.Crime_y)**2) + \
                w8*((row.Documentary_x - row.Documentary_y)**2) + \
                w9*((row.Drama_x - row.Drama_y)**2) + \
                w10*((row.Fantasy_x - row.Fantasy_y)**2) + \
                w12*((row.Horror_x - row.Horror_y)**2) + \
                w13*((row.IMAX_x - row.IMAX_y)**2) + \
                w14*((row.Musical_x - row.Musical_y)**2) + \
                w15*((row.Mystery_x - row.Mystery_y)**2) + \
                w16*((row.Romance_x - row.Romance_y)**2) + \
                w18*((row.Thriller_x - row.Thriller_y)**2) + \
                w19*((row.War_x - row.War_y)**2) + \
                w20*((row.Western_x - row.Western_y)**2)    
                  ) 
        tempvar = max(0, tempvar)
        tempvar = math.sqrt(tempvar)
        sum1 += tempvar
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity

In [14]:
def similarUsers(user):
    k = 25
    sim_scores = []
    for i in df.userId.unique():
        s = euclidean_similarity_weighted(df, user, i, user_weight_dict[user])
        #s = correlation(user,i)
        #s = euclidean_similarity(user,i)
        sim_scores.append(s)
    simusers = {'users':df.userId.unique(), 'similarity_score':sim_scores}
    simusers = pd.DataFrame(simusers)
    simusers = simusers.nlargest(k, 'similarity_score')
    return simusers

In [15]:
def predictRating(user,movie):
    n = 0
    predRating = 0
    k = 25
    #s = similarUsers(user)
    for i in similar_user['users']:
        rating = df[(df['userId']==i) & (df['movieId'] == movie)]['rating'].values
        sim = similar_user[similar_user['users']==i]['similarity_score'].values
        if(len(rating)!=0):
            predRating = predRating + (rating * sim)
    predRating = predRating/k 
    rating = df[df['userId']==user]['rating'].values
    average = np.average(rating)
    predRating = average+predRating
    return predRating

In [16]:
def getTopMovies(user):
    top_movies = []
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in df[df['userId']==user]['movieId'].values:
                if j not in top_movies:
                    top_movies.append(j)
    return top_movies

In [17]:
def getTopMovies1(user):
    top_movies = []
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in top_movies:
                top_movies.append(j)
    return top_movies

In [18]:
def recommendation(testUser):
    mo = []
    p = []
    u = []
    m = getTopMovies(testUser)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        u.append(testUser)
        #print(j)
        for i in predRating:
            if (np.isscalar(i)):
                p.append(i)
            else:
                p.append(i[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [19]:
def recommendation1(testUser):
    mo = []
    p = []
    u = []
    m = getTopMovies1(testUser)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        u.append(testUser)
        #print(j)
        for i in predRating:
            if (np.isscalar(i)):
                p.append(i)
            else:
                p.append(i[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [ ]:
for testUser in range(1,611):
    similar_user = similarUsers(testUser)
    df_u = recommendation(testUser)
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

recommendations for user 1 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



recommendations for user 2 :

Pulp Fiction (1994)
Schindler's List (1993)
Dances with Wolves (1990)
Pinocchio (1940)
Postman, The (Postino, Il) (1994)
Heavenly Creatures (1994)
In the Name of the Father (1993)
Beauty and the Beast (1991)
Once Were Warriors (1994)
Snow White and the Seven Dwarfs (1937)



recommendations for user 3 :

Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Alien (1979)
Indiana Jones and the Last Crusade (1989)
Jungle Book, The (1967)
North by Northwest (1959)
Termina

recommendations for user 27 :

Seven (a.k.a. Se7en) (1995)
Pulp Fiction (1994)
Schindler's List (1993)
Silence of the Lambs, The (1991)
Fargo (1996)
Reservoir Dogs (1992)
Apocalypse Now (1979)
Saving Private Ryan (1998)
Matrix, The (1999)
Fight Club (1999)



recommendations for user 28 :

Easy Rider (1969)
Matilda (1996)
10 Things I Hate About You (1999)
Muppet Treasure Island (1996)
Peter Pan (1953)
40-Year-Old Virgin, The (2005)
Clueless (1995)
Goofy Movie, A (1995)
Inception (2010)
Interstellar (2014)



recommendations for user 29 :

Toy Story (1995)
Heat (1995)
Leaving Las Vegas (1995)
Craft, The (1996)
Bed of Roses (1996)
James and the Giant Peach (1996)
Willy Wonka & the Chocolate Factory (1971)
Dead Man Walking (1995)
Mr. Holland's Opus (1995)
Nixon (1995)



recommendations for user 30 :

Rock, The (1996)
Rumble in the Bronx (Hont faan kui) (1995)
Chasing Amy (1997)
First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996)
Private Parts (19

recommendations for user 54 :

Star Wars: Episode IV - A New Hope (1977)
Rock, The (1996)
Star Wars: Episode VI - Return of the Jedi (1983)
Rumble in the Bronx (Hont faan kui) (1995)
Chasing Amy (1997)
First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996)
Private Parts (1997)
Barb Wire (1996)
Office Space (1999)
Star Wars: Episode I - The Phantom Menace (1999)



recommendations for user 55 :

Pulp Fiction (1994)
Monty Python's Life of Brian (1979)
Reservoir Dogs (1992)
Monty Python and the Holy Grail (1975)
Clockwork Orange, A (1971)
Goodfellas (1990)
Fight Club (1999)
Memento (2000)
Saw (2004)
Star Wars: Episode III - Revenge of the Sith (2005)



recommendations for user 56 :

Toy Story (1995)
Groundhog Day (1993)
Happy Gilmore (1996)
Phenomenon (1996)
Beavis and Butt-Head Do America (1996)
Nutty Professor, The (1996)
Mrs. Doubtfire (1993)
Truman Show, The (1998)
Incredibles, The (2004)
Finding Nemo (2003)



recommendations for user 57 :

Dar

recommendations for user 83 :

Usual Suspects, The (1995)
Schindler's List (1993)
Thin Red Line, The (1998)
Simpsons Movie, The (2007)
Awakenings (1990)
Big Chill, The (1983)
Ice Storm, The (1997)
Matrix, The (1999)
Patton (1970)
Godfather: Part III, The (1990)



recommendations for user 84 :

Seven (a.k.a. Se7en) (1995)
Dumbo (1941)
Fight Club (1999)
Memento (2000)
28 Days Later (2002)
Evil Dead II (Dead by Dawn) (1987)
Muppet Christmas Carol, The (1992)
Peter Pan (1953)
Shaun of the Dead (2004)
Evil Dead, The (1981)



recommendations for user 85 :

Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Bottle Rocket (1996)
Rob Roy (1995)
Canadian Bacon (1995)
Desperado (1995)
Billy Madison (1995)
Dumb & Dumber (Dumb and Dumber) (1994)
Star Wars: Episode IV - A New Hope (1977)
Tommy Boy (1995)



recommendations for user 86 :

Full Metal Jacket (1987)
Shining, The (1980)
Requiem for a Dream (2000)
Blade Runner (1982)
Chinatown (1974)
Third Man, The (1949)
Barton Fink (1991)
Stalker 

recommendations for user 111 :

Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Midnight Clear, A (1992)
Eight Men Out (1988)
Go (1999)
13th Warrior, The (1999)
Saint, The (1997)
World Is Not Enough, The (1999)
Player, The (1992)
Fargo (1996)
Leaving Las Vegas (1995)



recommendations for user 112 :

Schindler's List (1993)
Thin Red Line, The (1998)
Shawshank Redemption, The (1994)
Simpsons Movie, The (2007)
Awakenings (1990)
Big Chill, The (1983)
Ice Storm, The (1997)
Fight Club (1999)
Kingpin (1996)
Me, Myself & Irene (2000)



recommendations for user 113 :

Green Mile, The (1999)
Gladiator (2000)
Shawshank Redemption, The (1994)
Dark Knight, The (2008)
Dark Knight Rises, The (2012)
Girl with the Dragon Tattoo, The (2011)
Wolf of Wall Street, The (2013)
Whiplash (2014)
Life Is Beautiful (La Vita è bella) (1997)
Lion King, The (1994)



recommendations for user 114 :

Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Terminato

...

recommendations for user 165 :

Toy Story (1995)
Desperado (1995)
Clerks (1994)
Pulp Fiction (1994)
Tombstone (1993)
Shawshank Redemption, The (1994)
Like Water for Chocolate (Como agua para chocolate) (1992)
Babe (1995)
Apollo 13 (1995)
Quiz Show (1994)



recommendations for user 166 :

Beetlejuice (1988)
Casino (1995)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Midnight Clear, A (1992)
Eight Men Out (1988)
True Romance (1993)
Boot, Das (Boat, The) (1981)
James and the Giant Peach (1996)
Willy Wonka & the Chocolate Factory (1971)
Dead Man Walking (1995)



recommendations for user 167 :

Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Layer Cake (2004)
Lucky Number Slevin (2006)
Crash (2004)
Star Trek: First Contact (1996)
Matrix, The (1999)
Fight Club (1999)
Dark Knight, The (2008)
Minority Report (2002)



recommendations for user 168 :

Star Wars: Episode IV - A New Hope (1977)
Rock, The (1996)
Star Wars: Episode VI - Return of the Jed

recommendations for user 193 :

Pinocchio (1940)
James and the Giant Peach (1996)
Wizard of Oz, The (1939)
Sword in the Stone, The (1963)
Dumbo (1941)
Willy Wonka & the Chocolate Factory (1971)
E.T. the Extra-Terrestrial (1982)
Fantasia (1940)
Goonies, The (1985)
Jungle Book, The (1967)



recommendations for user 194 :

Toy Story (1995)
Usual Suspects, The (1995)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Jurassic Park (1993)
Silence of the Lambs, The (1991)
Independence Day (a.k.a. ID4) (1996)
Psycho (1960)
Blues Brothers, The (1980)
Terminator, The (1984)



recommendations for user 195 :

Easy Rider (1969)
Matilda (1996)
300 (2007)
10 Things I Hate About You (1999)
Muppet Treasure Island (1996)
Peter Pan (1953)
40-Year-Old Virgin, The (2005)
Dark Knight, The (2008)
Casablanca (1942)
Godfather, The (1972)



recommendations for user 196 :

Usual Suspects, The (1995)
Pulp Fiction (1994)
Forrest Gump (1994)
Fargo (1996)
Star Wars: Episode V - The Empire Strikes Back

recommendations for user 222 :

Toy Story (1995)
Fargo (1996)
James and the Giant Peach (1996)
Postman, The (Postino, Il) (1994)
Lone Star (1996)
Dead Man Walking (1995)
Sense and Sensibility (1995)
Leaving Las Vegas (1995)
Emma (1996)
Deep Impact (1998)



recommendations for user 223 :

This Is Spinal Tap (1984)
Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)
Star Trek: First Contact (1996)
American History X (1998)
Fight Club (1999)
Green Mile, The (1999)
Shutter Island (2010)
Inception (2010)
Interstellar (2014)
Truman Show, The (1998)



recommendations for user 224 :

Toy Story (1995)
Desperado (1995)
Clerks (1994)
Pulp Fiction (1994)
Tombstone (1993)
Shawshank Redemption, The (1994)
Like Water for Chocolate (Como agua para chocolate) (1992)
Babe (1995)
Apollo 13 (1995)
Quiz Show (1994)



recommendations for user 225 :

Star Wars: Episode IV - A New Hope (1977)
Fargo (1996)
Mighty Aphrodite (1995)
Postman, The (Postino, Il) (1994)
Everyone Says I Love You (1996)
Sense an

recommendations for user 250 :

Toy Story (1995)
Usual Suspects, The (1995)
Braveheart (1995)
Star Wars: Episode IV - A New Hope (1977)
Pulp Fiction (1994)
Forrest Gump (1994)
Jurassic Park (1993)
Schindler's List (1993)
Dances with Wolves (1990)
Batman (1989)



recommendations for user 251 :

E.T. the Extra-Terrestrial (1982)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Back to the Future (1985)
¡Three Amigos! (1986)
Star Wars: Episode I - The Phantom Menace (1999)
Hook (1991)
West Side Story (1961)
Gods Must Be Crazy, The (1980)
Babe (1995)
Mr. Holland's Opus (1995)



recommendations for user 252 :

From Dusk Till Dawn (1996)
Star Wars: Episode IV - A New Hope (1977)
Stargate (1994)
Silence of the Lambs, The (1991)
She's the One (1996)
Star Wars: Episode V - The Empire Strikes Back (1980)
Alien (1979)
Henry V (1989)
Pink Floyd: The Wall (1982)
Grosse Pointe Blank (1997)



recommendations for user 253 :

Seven (a.k.a. Se7en) (1995)
Usual Suspects, 

recommendations for user 278 :

Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Alien (1979)
Back to the Future (1985)
American History X (1998)
Total Recall (1990)
Green Mile, The (1999)
Gladiator (2000)



recommendations for user 279 :

Spirited Away (Sen to Chihiro no kamikakushi) (2001)
Producers, The (1968)
My Neighbor Totoro (Tonari no Totoro) (1988)
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Mystery Science Theater 3000: The Movie (1996)
Network (1976)
L.A. Confidential (1997)
Titanic (1997)
Sister Act (1992)
12 Angry Men (1957)



recommendations for user 280 :

Star Wars: Episode IV - A New Hope (1977)
Alien (1979)
Matrix, The (1999)
Event Horizon (1997)
Pulp Fiction (1994)
Schindler's List (1993)
Fight Club (1999)
Shawshank Redemption, The (1994)
Kill Bill: Vol. 1 (2003)
12 Angry Men (1957)



recommendations for user 281 :

Usual Suspects, The (199

In [ ]:
df_c = pd.DataFrame(columns=['userId','movieId','predicted rating'])
for testUser in df.userId.unique():
    similar_user = similarUsers(testUser)
    df_u = recommendation1(testUser)
    df_c = pd.concat([df_u, df_c])
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

In [ ]:
df_err = pd.merge(df,df_c,on=['movieId','userId'])
rmse = sqrt(mean_squared_error(df_err['rating'], df_err['predicted rating']))
print(rmse)

In [ ]:
ratings = []
predicted = []
for i in df_err['rating']:
    if(i>3.5):
        ratings.append(1)
    else:
        ratings.append(0)
for i in df_err['predicted rating']:
    if(i>3.5):
        predicted.append(1)
    else:
        predicted.append(0)

In [ ]:
from sklearn.metrics import f1_score
f1_score(ratings,predicted)